# PINN for Uniform Rectilinear Motion

# Importing Necessary Packages

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim

## Determining Loss Function

Below a Python function is developed to facilitate the development of our mathematical function, which afterwards will be used to define the network's loss function.   

Given we are dealing with a Uniform Rectilinear Motion (URM), the differential equation is given by

\begin{equation}
    \dfrac{dx}{dt} - v = 0,
\end{equation}

with boundary condition given by

\begin{equation}
    x(t=0) = 0
\end{equation}

where $x$ refers to a temporal function of position, $t$ to time and $v$ to a constant velocity. Basically, this equation describes the movement of a body with constant velocity.

For the purpose of this study, the domains considered will be

\begin{equation}
    t \in \left[ 0,1 \right]
\end{equation}

and

\begin{equation}
    v = 0.5.
\end{equation}

## Network's Architecture

In the following cell, we define the archictecture's hyperparameters, note that the activation function applied here is a hyperbolic tangent (Tanh) for it being non-linear and ranging from -1 to 1. It is also interesting to pay attention to the fact that we are building the linear neural network from scratch due the lack of a PyTorch module specific for it, i.e., we are building a custom module.

In [10]:
class LinearNN(nn.Module):
    def __init__(
            self,
            num_inputs: int=1,
            num_layers: int=1,
            num_neurons: int=5,
            act: nn.Module = nn.Tanh()
    ) -> None:
        super().__init__()
        self.num_inputs = num_inputs
        self.num_neurons = num_neurons
        self.num_layers = num_layers

        layers = []

        # Input layer
        layers.append(nn.Linear(self.num_inputs, num_neurons))

        # Hidden layers
        for _ in range(num_layers):
            layers.extend([nn.Linear(num_neurons, num_neurons), act])

        # Output layers
        layers.append(nn.Linear(num_neurons, 1))

        # Building the network as sequential
        self.network = nn.Sequential(*layers)

    # Setting up the output
    def forward(self, x:torch.Tensor) -> torch.Tensor:
        return self.network(x.reshape(-1,1)).squeeze()

## Loss Function

Here the loss function for our network will be defined, we begin by defining the boundary conditions $x0 = x(t=0)$ and $v=0.5$, afterwards a tensor of 100 elements is defined to receive the possible temporal coordinates that the model may assume.

Finally, the loss function will consider the model applied to the coordinates tensor and the function predicted at a given coordinate will be determined ($x_{pred}$). After determining the actual value of the function $x(t)$ at a given coordinate, the loss function will return the mean squared error between the predicted and actual values.

In [11]:
# Random list of 100 temporal cordinates in 1D
t = torch.rand(100, 1)

# Position coordinate of a border
x0 = 0
# Velocity
v = 0.5
# Function at the border
x = x0 + v*t

# Loss in the border
def loss_fn(model, t, x):
    x_pred = model(t)
    return torch.mean(torch.square(x - x_pred))

In [12]:
# Running the structure created
model = LinearNN()

epochs = 100

optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(epochs):

    optimizer.zero_grad()

    loss = loss_fn(model, t, x)
    loss.backward()

    print(f"Epoch {epoch} with loss {float(loss)}") 

Epoch 0 with loss 0.28474152088165283
Epoch 1 with loss 0.28474152088165283
Epoch 2 with loss 0.28474152088165283
Epoch 3 with loss 0.28474152088165283
Epoch 4 with loss 0.28474152088165283
Epoch 5 with loss 0.28474152088165283
Epoch 6 with loss 0.28474152088165283
Epoch 7 with loss 0.28474152088165283
Epoch 8 with loss 0.28474152088165283
Epoch 9 with loss 0.28474152088165283
Epoch 10 with loss 0.28474152088165283
Epoch 11 with loss 0.28474152088165283
Epoch 12 with loss 0.28474152088165283
Epoch 13 with loss 0.28474152088165283
Epoch 14 with loss 0.28474152088165283
Epoch 15 with loss 0.28474152088165283
Epoch 16 with loss 0.28474152088165283
Epoch 17 with loss 0.28474152088165283
Epoch 18 with loss 0.28474152088165283
Epoch 19 with loss 0.28474152088165283
Epoch 20 with loss 0.28474152088165283
Epoch 21 with loss 0.28474152088165283
Epoch 22 with loss 0.28474152088165283
Epoch 23 with loss 0.28474152088165283
Epoch 24 with loss 0.28474152088165283
Epoch 25 with loss 0.28474152088165

In [13]:
# Running the structure created
model = LinearNN()

# Python function to build mathematical functions
def f(x: torch.Tensor, params: dict({str: torch.nn.Parameter} )) -> torch.Tensor:
    return functional_call(model, params, (x, ))

# Mathematical function
dsdx = vmap(grad(f), in_dims=(0, None))

NameError: name 'vmap' is not defined

In [ ]:
# Setting the boundaries values
## Space boundary
S_BOUNDARY = 0.0
## Velocity boundary
C_BOUNDARY = 0.5

def loss_fn(params: torch.Tensor, x: torch.Tensor):

    # Interior equation
    interior = dsdx(x, params) - C_BOUNDARY
    
    x0 = S_BOUNDARY
    c0 = C_BOUNDARY
    # Parametrizing boundary
    s_boundary = torch.tensor([x0])
    c_boundary = torch.tensor([c0])
    
    # Boundary equation
    boundary = f(s_boundary, params) - c_boundary 

    # Setting Mean Squared Error as metric for loss
    loss = nn.MSELoss()
    # Getting numerical value for loss
    loss_value = loss(interior, torch.zeros_like(interior)) + loss(boundary, torch.zeros_like(boundary))

    return loss_value

In [ ]:
batch_size = 30
num_iter = 100
learning_rate = 1e-1
domain = (-5.0,5.0)

optmizer = torchopt.FuncOptimizer(torchopt.adam(lr=learning_rate))

params = tuple(model.parameters())

for i in range(num_iter):

    x = torch.FloatTensor(batch_size).uniform_(domain[0], domain[1])

    loss = loss_fn(params, x)

    params = optmizer.step(loss, params)

    print(f"Iteration {i} with loss {float(loss)}") 

ValueError: Expected all elements of parameter_and_buffer_dicts to be dictionaries